In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
path="/content/drive/MyDrive/b-11 major project/metadata_allvisits.csv"
df = pd.read_csv(path)
path1="/content/drive/MyDrive/b-11 major project/metadata_allvisitsfluency.csv"
df1 = pd.read_csv(path1)
path2="/content/drive/MyDrive/b-11 major project/metadata_allvisitsrecall.csv"
df2= pd.read_csv(path2)
path3="/content/drive/MyDrive/b-11 major project/metadata_allvisitssentence.csv"
df3= pd.read_csv(path3)

In [ ]:
df=pd.concat([df,df1, df2,df3])

In [ ]:
df['pauses']=df['pause1']+df['pause2']+df['pause3']
df=df.drop(['filepath','pause1','pause2','pause3'], axis = 1)

In [ ]:
df["mmse"].fillna(31, inplace = True)
df["age"].fillna(0, inplace = True)
df["gender"].fillna("other", inplace = True)

In [ ]:
df = df.dropna(axis=0, subset=['category'])
df["category"] = df["category"].str.lower()
df["category"] = df["category"].replace(to_replace ="probable",value ="probablead")
df["category"] = df["category"].replace(to_replace =["probablead","mci","memory","vascular","possiblead","dementia"],value =1)
df["category"] = df["category"].replace(to_replace =["control","other"],value = 0)
df.category.unique()

array([0, 1], dtype=int64)

In [ ]:
import string
import re
import pandas as pd
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
string_punctuation_1 = string.punctuation.replace(".","").replace("`","").replace("'","")
table_ = str.maketrans(string_punctuation_1, ' '*len(string_punctuation_1))
printable = set(string.printable)


def clean_data(text):

    sentence = text.lower()
    sentence_no_punct = sentence.translate(table_)
    space_remove = re.sub('\s+',' ', sentence_no_punct)
    return space_remove

In [ ]:
df['Transcripts_cleaned'] = df['data'].apply(lambda row: clean_data(row) )
df['Transcripts_cleaned'].iloc[1205]

" i like to write a note with a pencil . tree the tree is okay in the summertime but i don't like to crawl up in the wintertime . i i visit the child's hospital to see my neighbor the child . i i should have made it a little shorter but cold winter is so far we haven't had a cold winter . it isn't quite a winter yet . chair chairq doctorq and what oh sit in the doctor's chair . open the bureau door . "

In [ ]:
stemmer = SnowballStemmer('english')
def stem_words(text):
    text = text.split()
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

In [ ]:
df['Transcripts_stem'] = df['Transcripts_cleaned'].apply(lambda row: stem_words(row))
df['Transcripts_stem']

0      the scene is in the in the kitchen . the mothe...
1      oh i see the sink is run over . i see the stoo...
2      a boy and a girl are in the kitchen with their...
3      it was summertim and mother and the children w...
4      wait until i put my glass on . oh ‡ there a gi...
                             ...                        
236    pencilq . ‡ well you write with a pencil . wel...
237    treeq what is realli amaz me amaz me to me whe...
238    write me a letter . oh boy ‡ i'm way out today...
239    i write with a pencil . the tree are beauti in...
240    she wrote with a pencil . treeq . the tree is ...
Name: Transcripts_stem, Length: 1299, dtype: object

In [ ]:
# import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
import nltk
nltk.download('popular')

[nltk_data] Error loading popular: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [ ]:
lemmatizer = WordNetLemmatizer()

df['Transcripts_lem'] = df['Transcripts_cleaned'].apply(lambda row: " ".join([lemmatizer.lemmatize(i) for i in word_tokenize(row)]))

df['Transcripts_lem']

ImportError: cannot import name 'punkt' from 'nltk.corpus' (D:\python\Lib\site-packages\nltk\corpus\__init__.py)

In [ ]:
# df["labels"] = df["category"].astype('category')
# df.dtypes
# df["AD"] = df["labels"].cat.codes
# df.labels.unique()
# df["category"] = df["category"].replace(to_replace =["probablead","mci","memory","vascular","possiblead","dementia"],value =1)
# df["category"] = df["category"].replace(to_replace =["control","other"],value = 0)
df.to_csv('/content/drive/MyDrive/Major Project stuff/metadata.csv')

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/Major Project stuff/metadata.csv')

In [ ]:
df

In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch-pretrained-bert

In [ ]:
from transformers import BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
sentences = df['Transcripts_lem'].values
labels = df.category.values

In [ ]:
max_len = 0

for sent in sentences:
    input_ids = tokenizer.encode(sent, add_special_tokens=True,max_length=256)
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
import tensorflow as tf
import torch

In [ ]:
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 256,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

In [ ]:
print(input_ids.shape)
print(attention_masks.shape)
print(labels.shape)

In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size,test_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )
test_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset),
            batch_size = batch_size
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Tell pytorch to run this model on the GPU.
model.cuda()

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 4

total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
device = torch.device("cuda")

In [ ]:
import random
import numpy as np

# Setting the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


training_stats = []
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpacking this training batch from our dataloader.
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        # Performing a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss=outputs[0]

        # Accumulating the training loss over all of the batches so that we can
        total_train_loss += loss.item()

        # Performing a backward pass to calculate the gradients.
        loss.backward()

        # Clipping the norm of the gradients to 1.0.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        # Updating the learning rate.
        scheduler.step()

    # Calculating the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measuring how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():

            # Forward pass, calculating logit predictions.
            outputs = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss=outputs[0]

        # Accumulating the validation loss.
        total_eval_loss += loss.item()

        logits=outputs[1]
        # Moving logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculating the accuracy for this batch of test sentences, and accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # Reporting the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculating the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measuring how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Recording all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
import pandas as pd

pd.set_option('precision', 6)

df_stats = pd.DataFrame(data=training_stats)

df_stats = df_stats.set_index('epoch')

df_stats

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

sns.set(style='ticks')

sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

plt.plot(df_stats['Training Loss'], 'b', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g', label="Validation")

plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

sns.set(style='darkgrid')

sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

plt.plot(df_stats['Valid. Accur.'], 'b', label="Validation_acc")
plt.plot(df_stats['Valid. Loss'], 'g', label="Validation_loss")

plt.title("Validation accuracy & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()


In [ ]:
import torch.nn.functional as F
# Put model in evaluation mode
model.eval()

predictions , true_labels = [], []
prediction_probs = []
for batch in test_dataloader:
  batch = tuple(t.to(device) for t in batch)

  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)

  _, preds = torch.max(outputs[0], dim=1)
  logits = outputs[0]
  predictions.extend(preds)
  prediction_probs.extend(outputs[0])

  logits = logits.detach().cpu().numpy()

  #predictions.append(logits)
  true_labels.extend(b_labels)
predictions = torch.stack(predictions).cpu()
prediction_probs = torch.stack(prediction_probs).cpu()
true_labels = torch.stack(true_labels).cpu()

print('    DONE.')

In [ ]:
true_labels

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.metrics import roc_curve , roc_auc_score, auc

def evaluate_roc(probs, y_true):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    preds = probs[:, 1]
    fpr, tpr, threshold = roc_curve(y_true, preds)
    roc_auc = auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')

    y_pred = np.where(preds >= 0.5, 1, 0)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')


    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
evaluate_roc(prediction_probs , true_labels)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.metrics import roc_curve , roc_auc_score


labels = [0, 1]
print(accuracy_score(true_labels, predictions))
print(confusion_matrix(true_labels, predictions))
# print(precision_recall_fscore_support(true_labels, predictions, average=None, labels= labels))
print(classification_report(true_labels, predictions))